# Análisis de datos de un dataset usando la librería Pandas de Python
En este artículo vamos a ver un ejemplo de análisis de datos mediante la librería Pandas de Python.

Se va a ver los distintos procesos que se require realizar para llevar a cabo esta tarea, ya que no solo consiste en analizar los datos sino que abarca otros procesos igual de importante como son la carga de los datos y limpiar los datos. 
Estos dos procesos: obtención de los datos y limpieza de los mismos son procesos que son necesarios también en el Machine Learning (de hecho suponen cerca del 80% del tiempo empleado), por lo que son procesos muy importantes y que deben realizarse con cuidado ya que la calidad del resultado del análisis o del modelo generado en el caso de Machine Learning dependen en gran medida de la calidad de los datos

El artículo se ha realizado en un notebook de Jupyter para que tanto el código como la documentación esten en un mismo sitio. El código puede ser ejecutado desde una terminal de forma normal (se puede extrar el código del notebook de forma sencilla desde la interfaz de Jupyter)
Para instalar Jupyter puede serguir las instrucciones indicadas en la web oficial: https://jupyter.org/install o usar docker: https://github.com/jupyter/docker-stacks

También se puede usar un servicio online como https://colab.research.google.com/ si no se desea instalar nada.

# 1. Caso de estudio y objetivo del análisis
Para nuestro ejemplo se va a utilizar los datos que provee la AMAET (Agencia estatal de Meteorologia). 
Se ha usado los enlaces disponibles en la web https://datosclima.es/Aemet2013/DescargaDatos.html ya que es mas sencillo su descarga a través de esa web

Los datos contienen información diaria sobre temperatura (máxima, mínima y media), viento (racha y velocidad máxima de las medias) y precipitaciones (total y por tramo horario) para cada estación meteorologica de España.

En algunos casos falta información y eso se tiene que tener en cuenta durante el proceso de limpieza de datos.

Se ha analizado únicamente el mes de Enero por simplificar el dataset.

El análisis que se va a realizar en este artículo pretende ser simplemente un ejemplo de lo sencillo que es realizar operaciones sobre los datos mediante la librería Pandas y va a consistir en obtener ciertos valores para tener una visión del clima en las distintas provincias de España. No pretende ser un análisis completo de los datos meteorológicos.

# 2. Pandas y Python
Se va a usar el lenguaje Python ya que es probablemente el lenguaje más usado junto a R para el análisis científico de datos, asi como para el Machine Learning

La version de Python usada es la 3 (https://www.python.org/downloads/)

Pandas (https://pandas.pydata.org) es la librería de facto para el tratamiento y análisis de datos cuando usamos Python
Nos proporciona la clase Dataframe la cual permite almacenar los datos en un objeto similar a una tabla de una base de datos.
Lo que hace a esta librería ser realmente útil para el análisis de datos son los métodos que nos proporciona, los cuales nos permiten realizar tareas de ordenación, agrupación, etc de forma sencilla y rápida como veremos.

Se han utilizado otras librerias de python para ciertas tareas

Todas las librerias se han instalado mediante el gestor de librerias de python: pip

# 3. Implementación

## 3.1 Importación de las librerias necesarias
Importamos las librerías que vamos a usar:
- **Pandas:** libería principal para la carga y el análisis de datos
- **Glob:** librería para la gestión de ficheros. En nuestro caso se usa únicamente para realizar la carga de varios ficheros de forma más sencilla
- **Numpy:** nos proporciona métodos de gestión de arrays

In [32]:
#!pip install pandas
#!pip install numpy
import pandas as pd
import glob
import numpy as np

### Carga de datos
Pandas nos permite obtener los datos de diversas fuentes, principalmente ficheros. Además, podemos leer directamente de distintos formatos de fichero: xls, csv, json, etc. En nuestro caso leeremos ficheros xls.

Es importante también tener en cuenta que se puede escribir en fichero el contenido de los datos, es decir, podemos leer los datos, modificarlos y escribirlos de nuevo en fichero. De esta forma podemos hacer procesos de modificación de datos de forma muy sencilla

En nuestro caso, obtenemos los ficheros rar que contienen las hojas de cálculo con los datos. Cada fichero rar contiene los ficheros xls correspondientes a cada día de ese mes. Para nuestro análisis hemos cogido solo 1 mes (Enero), pero sería igual de sencillo cargar datos de otros meses.

In [33]:
!wget https://datosclima.es/capturadatos/Aemet2019-01.rar

--2020-07-09 18:10:33--  https://datosclima.es/capturadatos/Aemet2019-01.rar
Resolving datosclima.es (datosclima.es)... 95.217.108.87
Connecting to datosclima.es (datosclima.es)|95.217.108.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1456378 (1.4M) [application/x-rar-compressed]
Saving to: ‘Aemet2019-01.rar.9’

Aemet2019-01.rar.9  100%[===================>]   1.39M  3.00MB/s    in 0.5s    

2020-07-09 18:10:33 (3.00 MB/s) - ‘Aemet2019-01.rar.9’ saved [1456378/1456378]



In [34]:
# Instala unrar si es necesario
!unrar e -o+ Aemet2019-01.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Aemet2019-01.rar

Extracting  Aemet2019-01-22.xls                                            3  OK 
Extracting  Aemet2019-01-23.xls                                            6  OK 
Extracting  Aemet2019-01-24.xls                                           10  OK 
Extracting  Aemet2019-01-25.xls                                         1 13  OK 
Extracting  Aemet2019-01-26.xls                                         1 16  OK 
Extracting  Aemet2019-01-27.xls                                         1 19  OK 
Extracting  Aemet2019-01-28.xls                                         2 23  OK 
Extracting  Aemet2019-01-29.xls                                         2 26  OK 
Extracting  Aemet2019-01-30.xls                                         2 30  OK 
Extracting  Aemet2019-01-31.xls                                         3 33  OK 
Extracting  Aemet2019-01-01.xls                                         36  O

Recorremos los ficheros xls ya descomprimidos y los vamos cargando en un objeto dataframe.
Si vemos el contenido de los ficheros xls podemos observar que no contienen la fecha de los datos, lo cual es un dato importante porque queremos ordenar los datos por fecha en algunos casos. Para obtener este dato vamos a usar el nombre del fichero, el cual contiene la fecha a la que corresponden los datos. Vamos a incorporar una columa adicional al dataset con el nombre del fichero que estamos leyendo:

In [35]:
all_files = glob.glob("Aemet2019-*.xls")

file_list = []
for f in all_files:
    data = pd.read_excel(f,skiprows=4)
    data['source_file'] = f # nueva columna conteniendo el nombre del fichero leido
    file_list.append(data)
df = pd.concat(file_list)

Vamos a ver cuantas filas y columnas contiene nuestro dataset:

In [36]:
df.shape

(24707, 13)

Vemos que tiene 24707 filas y 13 columnas

Veamos las 5 primeras filas para ver como son los datos:

In [37]:
df.head()

,Estación,Provincia,Temperatura máxima (ºC),Temperatura mínima (ºC),Temperatura media (ºC),Racha (km/h),Velocidad máxima (km/h),Precipitación 00-24h (mm),Precipitación 00-06h (mm),Precipitación 06-12h (mm),Precipitación 12-18h (mm),Precipitación 18-24h (mm),source_file
0,Estaca de Bares,A Coruña,11.2 (15:30),10.0 (03:40),10.6,64 (22:20),49 (22:20),1.6,0.0,1.6,0.0,0.0,Aemet2019-01-09.xls
1,As Pontes,A Coruña,10.7 (14:00),4.6 (05:30),7.6,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Aemet2019-01-09.xls
2,A Coruña,A Coruña,13.1 (12:30),7.4 (02:40),10.3,41 (18:20),28 (15:40),0.0,0.0,0.0,0.0,0.0,Aemet2019-01-09.xls
3,A Coruña Aeropuerto,A Coruña,12.6 (15:50),-0.8 (05:40),5.9,35 (15:20),23 (15:20),0.0,0.0,0.0,0.0,0.0,Aemet2019-01-09.xls
4,"Carballo, Depuradora",A Coruña,11.6 (13:40),-1.4 (03:20),5.1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Aemet2019-01-09.xls


## 3.2 Limpieza de datos
Una parte importante del proceso de obtención de datos es limpiar su contenido para que sea adecuado a nuestras necesidades y para hacerlos mas cómodos de usar

Podemos empezar renombrando las columnas para que sea mas sencillo usarlas y ocupen menos espacio en pantalla

In [38]:
df=df.rename(columns={'Estación': 'estacion','Provincia':'provincia','Temperatura máxima (ºC)':'temp_max','Temperatura mínima (ºC)':'temp_min','Temperatura media (ºC)':'temp_med','Racha (km/h)':'viento_racha','Velocidad máxima (km/h)':'viento_vel_max','Precipitación 00-24h (mm)':'prec_dia','Precipitación 00-06h (mm)':'prec_0_6h','Precipitación 06-12h (mm)':'prec_6_12h','Precipitación 12-18h (mm)':'prec_12_18h','Precipitación 18-24h (mm)':'prec_18_24h','source_file':'fecha'})
df.head(1)

,estacion,provincia,temp_max,temp_min,temp_med,viento_racha,viento_vel_max,prec_dia,prec_0_6h,prec_6_12h,prec_12_18h,prec_18_24h,fecha
0,Estaca de Bares,A Coruña,11.2 (15:30),10.0 (03:40),10.6,64 (22:20),49 (22:20),1.6,0.0,1.6,0.0,0.0,Aemet2019-01-09.xls


Podemos observar que en algunas columnas ademas del valor numérico también se tiene la hora, lo cual no nos interesa ya que los datos tienen que ser numéricos para nuestro análisis. Asi que vamos a quitar esa parte de los datos

In [39]:
df=df.replace(to_replace=r'.\(.+\)$', value='', regex=True)
df.head(1)

,estacion,provincia,temp_max,temp_min,temp_med,viento_racha,viento_vel_max,prec_dia,prec_0_6h,prec_6_12h,prec_12_18h,prec_18_24h,fecha
0,Estaca de Bares,A Coruña,11.2,10.0,10.6,64,49,1.6,0.0,1.6,0.0,0.0,Aemet2019-01-09.xls


La columna que hemos creado para disponer del nombre del fichero y asi disponer de la fecha de los datos contiene más texto además de la fecha. Vamos a limpiar el valor para dejar solo la fecha

In [40]:
df.fecha=df.fecha.replace(regex={'Aemet':'',r'\.+xls':''})
df.head(1)

,estacion,provincia,temp_max,temp_min,temp_med,viento_racha,viento_vel_max,prec_dia,prec_0_6h,prec_6_12h,prec_12_18h,prec_18_24h,fecha
0,Estaca de Bares,A Coruña,11.2,10.0,10.6,64,49,1.6,0.0,1.6,0.0,0.0,2019-01-09


Vamos a ver de que tipo son las columnas actualmente. El tipo se puede definir explícitamente al leer el fichero, pero para este artículo hemos dejado que Pandas lo calcule automáticamente

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24707 entries, 0 to 796
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   estacion        24707 non-null  object 
 1   provincia       24707 non-null  object 
 2   temp_max        23650 non-null  object 
 3   temp_min        23650 non-null  object 
 4   temp_med        23650 non-null  float64
 5   viento_racha    20072 non-null  object 
 6   viento_vel_max  20203 non-null  object 
 7   prec_dia        23366 non-null  float64
 8   prec_0_6h       23565 non-null  float64
 9   prec_6_12h      23565 non-null  float64
 10  prec_12_18h     23584 non-null  float64
 11  prec_18_24h     23563 non-null  float64
 12  fecha           24707 non-null  object 
dtypes: float64(6), object(7)
memory usage: 2.6+ MB


Vemos que hay columnas con tipo "object" cuando debería ser un float (por ejemplo: temp_max y viento_racha). Esto es porque hay filas que no tienen valor para esas columnas, luego veremos que hacer con ellas.
Por ahora vamos a poner los tipos adecuados a cada columna

In [42]:
df=df.astype({'estacion': 'string','provincia': 'string', 'temp_max':'float64', 'temp_min':'float64', 'viento_racha':'float64', 'viento_vel_max':'float64','fecha': 'datetime64[ns]'})
df['fecha']=pd.to_datetime(df["fecha"].dt.strftime('%Y-%m-%d'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24707 entries, 0 to 796
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   estacion        24707 non-null  string        
 1   provincia       24707 non-null  string        
 2   temp_max        23650 non-null  float64       
 3   temp_min        23650 non-null  float64       
 4   temp_med        23650 non-null  float64       
 5   viento_racha    20072 non-null  float64       
 6   viento_vel_max  20203 non-null  float64       
 7   prec_dia        23366 non-null  float64       
 8   prec_0_6h       23565 non-null  float64       
 9   prec_6_12h      23565 non-null  float64       
 10  prec_12_18h     23584 non-null  float64       
 11  prec_18_24h     23563 non-null  float64       
 12  fecha           24707 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(10), string(2)
memory usage: 2.6 MB


Vamos a eliminar las estaciones que tienen filas a las que les faltan datos para estandarizar los datos. 
Realmente habria que intentar rellenar los datos que faltan mediante alguna de las técnicas recomendadas (media, propagate last valid observation forward, etc) pero para nuestro ejemplo vamos a simplemente eliminar filas por simplificar

In [43]:
estaciones_con_nan=df[df.isnull().any(axis=1)]['estacion'].unique()
df.drop(df[df['estacion'].isin(estaciones_con_nan)].index, inplace=True)
df.isna().sum()

estacion          0
provincia         0
temp_max          0
temp_min          0
temp_med          0
viento_racha      0
viento_vel_max    0
prec_dia          0
prec_0_6h         0
prec_6_12h        0
prec_12_18h       0
prec_18_24h       0
fecha             0
dtype: int64

## 3.3 Análisis de datos

Ahora que ya tenemos los datos limpios vamos a empezar a analizarlos, demostrando que fácil es realizarlo con pandas

### Primeros contactos con los datos 

Vamos a usar el método "describe" para ver de forma sencilla algunas medidas de nuestros datos.
También es un ejemplo de como podemos seleccionar ciertas columnas por su indice mediante el uso de la librería numpy

In [44]:
df.iloc[:,np.r_[0:8,12:13]].describe()

,temp_max,temp_min,temp_med,viento_racha,viento_vel_max,prec_dia
count,14663.000000,14663.000000,14663.000000,14663.000000,14663.000000,14663.000000
mean,12.558869,2.557737,7.560342,34.010434,19.691741,1.490009
std,4.506435,5.256177,4.350412,17.793154,11.139909,5.890856
min,-5.000000,-20.500000,-7.400000,0.000000,0.000000,0.000000
25%,9.500000,-1.300000,4.500000,20.000000,12.000000,0.000000
50%,12.700000,2.100000,7.300000,30.000000,17.000000,0.000000
75%,15.700000,5.900000,10.200000,45.000000,26.000000,0.000000
max,27.800000,19.100000,21.100000,146.000000,113.000000,132.800000


Ahora vamos a mostrar 5 filas al azar como muestreo de los datos. 
También es un ejemplo de selección de columnas por expresion regular (seleccionamos las que empiezan por el prefijo "temp_"

In [45]:
df.filter(regex='^temp_*', axis=1).sample(5)

,temp_max,temp_min,temp_med
36,19.8,3.8,11.8
285,8.3,5.7,7.0
642,8.5,-4.7,1.9
48,17.4,12.4,14.9
729,11.8,2.5,7.2


### Medias por provincia

Vamos a mostrar a continuación la media de los valores de cada columna agrupando por provincia.
Podemos observar que el método es lo suficientemente inteligente para mostrarnos solo columnas de tipo numérico

In [46]:
df.groupby('provincia').mean()

,temp_max,temp_min,temp_med,viento_racha,viento_vel_max,prec_dia,prec_0_6h,prec_6_12h,prec_12_18h,prec_18_24h
provincia,,,,,,,,,,
A Coruña,12.402765,5.505991,8.958065,43.552995,27.073733,3.329954,0.540553,1.136866,0.994009,0.658525
Alacant/Alicante,15.780645,3.768664,9.777880,32.852535,18.253456,0.070968,0.031336,0.003687,0.017512,0.018433
Albacete,12.548387,-0.327419,6.117204,32.806452,17.376344,0.114516,0.031183,0.024194,0.007527,0.051613
Almería,16.347312,7.915591,12.132258,40.731183,27.166667,0.101613,0.061828,0.007527,0.005376,0.026882
Araba/Álava,7.577419,1.049194,4.316129,32.395161,17.387097,5.145161,1.417742,1.216129,0.879032,1.632258
Asturias,11.186380,4.294982,7.743011,32.949821,18.232975,6.780287,1.739068,1.463441,1.688530,1.889247
Badajoz,13.981774,2.197742,8.091935,27.193548,17.250000,0.546935,0.099677,0.061452,0.080645,0.305161
Barcelona,11.641505,1.374409,6.509462,29.066667,16.169892,0.471183,0.138925,0.254409,0.015269,0.062581
Bizkaia,9.935484,4.251613,7.095699,35.903226,20.612903,9.092473,2.565054,2.340860,1.703226,2.483333


### Mayores temperaturas maximas por estación 

Vamos a mostrar los 3 valores mas altos de temperatura máxima que se han registrado en el mes.
Es un ejemplo de como ordenar por columna, mostrar solo las columnas que nos interesa y de esas columnas mostramos solo las primeras 3 filas

In [47]:
df.sort_values(by='temp_max',ascending=False)[['fecha','estacion','provincia','temp_max']].head(3)

,fecha,estacion,provincia,temp_max
437,2019-01-13,La Aldea de San Nicolás,Las Palmas,27.8
427,2019-01-26,Pájara,Las Palmas,25.7
437,2019-01-01,La Aldea de San Nicolás,Las Palmas,25.6


### Menores temperaturas minimas por estación  

De forma similar obtenemos los 3 valores más bajos de temperatura minima

In [48]:
df.sort_values(by='temp_min',ascending=True)[['fecha','estacion','provincia','temp_min']].head(3)

,fecha,estacion,provincia,temp_min
522,2019-01-21,Villanueva de la Cañada,Madrid,-20.5
639,2019-01-06,Cuéllar,Segovia,-12.1
763,2019-01-06,Sardón de Duero,Valladolid,-11.8


### Maximas diferencias entre temperatura max y min por provincia

Ahora calculamos una nueva columna restando de la temperatura máxima la temperatura mínima. Vemos que simplemente restamos columnas. Pandas es intuitivo.

In [49]:
df['diff_temp_min_max']=df['temp_max']-df['temp_min']
df.iloc[df.reset_index().groupby('provincia')['diff_temp_min_max'].idxmax()].sort_values(by='diff_temp_min_max',ascending=False)[['fecha','estacion','provincia','diff_temp_min_max','temp_min','temp_max']].head(10)

,fecha,estacion,provincia,diff_temp_min_max,temp_min,temp_max
522,2019-01-21,Villanueva de la Cañada,Madrid,30.9,-20.5,10.4
763,2019-01-06,Sardón de Duero,Valladolid,26.6,-11.8,14.8
639,2019-01-06,Cuéllar,Segovia,26.6,-12.1,14.5
272,2019-01-05,Salvacañete,Cuenca,25.6,-8.8,16.8
32,2019-01-01,Villena,Alacant/Alicante,25.5,-6.2,19.3
665,2019-01-15,Burgo de Osma,Soria,25.3,-7.6,17.7
299,2019-01-05,La Vall de Bianya,Girona,25.0,-4.4,20.6
317,2019-01-15,Molina de Aragón,Guadalajara,24.9,-8.5,16.4
223,2019-01-06,"Valderredible, Cubillo de Ebro",Cantabria,24.3,-11.3,13.0
472,2019-01-02,Martinet,Lleida,24.1,-6.2,17.9


### Mayores rachas de viento en un día por provincia
Vemos ahora un ejemplo de agrupar por una columna (provincia), obtener el máximo de los valores de otra columna (viento_racha) para cada grupo y ordenar posteriormente esas filas. Todo se realiza en una sola línea

In [50]:
df.iloc[df.reset_index().groupby('provincia')['viento_racha'].idxmax()].sort_values(by='viento_racha',ascending=False)[['fecha','estacion','provincia','viento_racha']].head(5)

,fecha,estacion,provincia,viento_racha
0,2019-01-29,Estaca de Bares,A Coruña,146.0
139,2019-01-29,Machichaco,Bizkaia,144.0
231,2019-01-27,La Pobla de Benifassà-Fredes,Castelló/Castellón,129.0
370,2019-01-27,"Sierra de Alfabia, Bunyola",Illes Balears,127.0
302,2019-01-25,Espolla,Girona,111.0


### Mayores velocidades de viento medio en un día por provincia

Hacemos de forma similar el cálculo de las mayores velocidades de viento medio en un día por provincia.

In [51]:
df.iloc[df.reset_index().groupby('provincia')['viento_vel_max'].idxmax()].sort_values(by='viento_vel_max',ascending=False)[['fecha','estacion','provincia','viento_vel_max']].head(5)

,fecha,estacion,provincia,viento_vel_max
139,2019-01-23,Machichaco,Bizkaia,113.0
0,2019-01-29,Estaca de Bares,A Coruña,103.0
726,2019-01-28,Andorra,Teruel,82.0
688,2019-01-17,Izaña,Santa Cruz de Tenerife,80.0
46,2019-01-28,Láujar de Andarax,Almería,78.0


### Maximas velocidades de viento por provincia de media
Vemos un ejemplo de calcular una media por grupo de la forma mas fácil e intuitiva posible (con "group" y "mean").

In [52]:
df.groupby('provincia').mean()[['viento_vel_max']].sort_values(by='viento_vel_max',ascending=False).head(5)

,viento_vel_max
provincia,
Castelló/Castellón,27.669355
Almería,27.166667
A Coruña,27.073733
Teruel,26.210918
Soria,25.991935


### Maximas diferencias de temperatura maxima entre días
Vamos a ver un ejemplo de como podemos hacer una operación (restar) entre la fila actual y la fila anterior de forma sencilla: con un "shift" desplazamos una columna hacia abajo.

In [53]:
df['temp_max_shifted']=df.groupby('estacion')['temp_max'].transform('shift')
df['dif_prev_temp_max']=df.temp_max-df.temp_max_shifted

df.iloc[df.reset_index().groupby('provincia')['dif_prev_temp_max'].idxmax()].sort_values(by='dif_prev_temp_max',ascending=False)[['fecha','estacion','provincia','dif_prev_temp_max']].head(5)


,fecha,estacion,provincia,dif_prev_temp_max
505,2019-01-01,Rascafría,Madrid,16.3
605,2019-01-01,"Velilla del Río Carrión, Camporredondo de Alba",Palencia,15.5
470,2019-01-01,Lagunas de Somoza,León,14.5
299,2019-01-05,La Vall de Bianya,Girona,13.6
343,2019-01-05,Aragüés del Puerto,Huesca,13.6


### Racha de días seguidos lloviendo por provincia y estación
El proceso de obtener rachas, es decir, número de días en los que pasa un suceso (en este caso llover) consecutivamente es un proceso algo más elaborado pero que se puede realizar en unas pocas lineas de código aprovechando las siguientes funcionalidades del dataframe:
- Creación de una nueva columna cuyo valor se obtiene a partir de otra columna aplicando una función lambda (cálculo de la columna "llueve")
- Comparación de valores (operador "ne") entre columnas de una misma fila
- Desplazamiento (método "shift") de una columna para disponer de datos anteriores en la misma fila
- Generación de un número secuencial (identificador) de los elementos de cada grupo (método "cumsum")
- Cálculo del número de elementos de un grupo (método "cumcount")
- Asignación de un valor a una columna para las filas que cumplan una condición (llueve==0)

In [54]:
df['llueve']=df.prec_dia.apply(lambda x: 1 if x>0 else 0)

df.sort_values(by=['estacion','fecha'],ascending=True,inplace=True,ignore_index=True)

df['start_of_streak'] = df.llueve.ne(df['llueve'].shift())

df['estacion_anterior']=df['estacion'].shift()
df.loc[(df['estacion'].ne(df['estacion_anterior'])), 'start_of_streak'] = True

df['streak_id'] = df['start_of_streak'].cumsum()
df['dias_seguidos_lloviendo'] = df.groupby('streak_id').cumcount() + 1
df.loc[(df['llueve']==0), 'dias_seguidos_lloviendo'] = 0

df.iloc[df.reset_index().groupby(['provincia','estacion'])['dias_seguidos_lloviendo'].idxmax()].sort_values(by='dias_seguidos_lloviendo',ascending=False)[['estacion','provincia','dias_seguidos_lloviendo']].head(10)

,estacion,provincia,dias_seguidos_lloviendo
5083,Elgeta,Gipuzkoa,25
14631,Zumarraga,Gipuzkoa,20
5455,Forua,Bizkaia,18
9795,Orozko,Bizkaia,18
7966,Lugo,Lugo,16
7563,Llanes,Asturias,16
30,A Coruña,A Coruña,16
11903,Santiago de Compostela,A Coruña,16
7904,"Los Tojos, Bárcena Mayor",Cantabria,15
9702,Ordizia,Gipuzkoa,15


### Donde llueve menos días seguidos por provincia y estación

Aprovechando la nueva columna "dias_seguidos_lloviendo" podemos sacar esta nueva métrica

In [55]:
df.iloc[df.reset_index().groupby(['provincia','estacion'])['dias_seguidos_lloviendo'].idxmax()].sort_values(by='dias_seguidos_lloviendo',ascending=True)[['estacion','provincia','dias_seguidos_lloviendo']].head(10)

,estacion,provincia,dias_seguidos_lloviendo
7781,"Lorca, Zarcilla de Ramos",Murcia,0
2542,Belmonte,Cuenca,0
12617,Tazacorte,Santa Cruz de Tenerife,0
1116,Almería Aeropuerto,Almería,0
434,Albox,Almería,0
3100,Buñol,València/Valencia,0
8525,"Mogán, Puerto Rico",Las Palmas,0
13595,Vilafranca del Penedès,Barcelona,1
13473,"València, Viveros",València/Valencia,1
7580,Lleida,Lleida,1


### Mostrar cuantas estaciones hay por provincia y un total general

Vamos a ver un ejemplo de que podemos tener copias "temporales" del dataframe para hacer operaciones sin destruir el original.
Queremos obtener el número de estaciones meteorológicas que hay por cada provincia. 
Para ello nos quedamos con solo las columnas relevantes: provincia y estacion.
Luego eliminamos las filas duplicadas para tener los pares únicos de provincia-estacion.
Ya nos queda simplemente agrupar por provincia y sacar el número de filas por cada grupo.
Todo eso hacemos en 1 sola línea de código.

Finalmente añadimos otra fila adicional al resultado con el total de estaciones (un SUM tipico de Excel)

In [56]:
df_estaciones=df[['provincia','estacion']].drop_duplicates().groupby('provincia').count()
df_estaciones.loc['Total de estaciones'] = df_estaciones['estacion'].sum()
df_estaciones

,estacion
provincia,
A Coruña,7
Alacant/Alicante,7
Albacete,6
Almería,6
Araba/Álava,4
Asturias,9
Badajoz,20
Barcelona,15
Bizkaia,6


# 4. Conclusiones

Hemos visto como es muy sencillo realizar tareas de carga de datos, limpieza de datos y extracción de información usando Pandas.
Las operaciones que haríamos en una base de datos (agrupar, calcular máximos y mínimos) e incluso algunas tareas más complejas como obtener rachas se hacen de forma sencilla, rápida y lo que es importante de forma programática. Esto último es importante porque el código puede ser repositorizado, trackeado y ejecutado de forma automatizada.

Vemos también que se puede sacar información relevante de cualquier dataset. No es necesario llegar a usar algoritmos de Machine Learning para sacar provecho de la información disponible. La explotación de los datos mediante librerias como Pandas hace que podamos extraer información valiosa de forma sencilla y rápida, algo que es sumamente importante para cualquier empresa, entidad o incluso a nivel particular. Podemos verlo como una mezcla entre Excel y SQL pero con las ventajas que hemos ido mostrando en este artículo.

Espero que os haya gustado este primer contacto con la liberia Pandas y estéis deseando probarla. 
Es una librería ampliamente usada para las tareas de Machine Learning y Data Science pero es realmente útil para cualquier persona que tenga que tratar con datos.

Tambien espero que haya servido como primer contacto con los notebook de Jupyter, que son una herramienta increible para tener en un mismo sitio el código y la documentación. Ademas de que permiten compartirlo de forma sencilla y ser un punto de partida para la experimentación con el código contenido.